<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/635_MOv2_DataGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is a **Marketing Orchestrator V2 — Data Schema Proposal** that adds *just enough* realism to unlock new orchestration behaviors (journey-aware optimization, budget moves, risk escalation) while staying MVP-simple and aligned to your existing V1 objects: campaigns , segments , channels , assets , experiments , performance , decisions , ROI ledger .

---

# Marketing Orchestrator V2 — Data Schema Proposal

## V2 goals

1. **Add 2–4 new datasets** (small, composable).
2. Keep everything **joinable via your existing IDs** (`campaign_id`, `segment_id`, `channel_id`, `asset_id`, `experiment_id`).
3. Enable new V2 behaviors:

   * Journey-stage optimization
   * Budget reallocations with approvals
   * Risk signals + escalation triggers
   * Segment rollups for executive reporting

---

## Existing V1 datasets (no change required)

These are solid—keep as-is:

* `campaigns.json`
* `audience_segments.json`
* `channels.json`
* `creative_assets.json`
* `experiments.json`
* `performance_metrics.json`
* `orchestrator_decisions.json`
* `roi_ledger.json`

---

# NEW V2 datasets

## 1) `funnel_events.json` (journey-stage signals)

### Why add it

Right now you see results (impressions/clicks/conversions) , but you don’t have a simple “where are people getting stuck?” view. Funnel events give you journey-stage reasoning without needing a full attribution system.

### Grain

**(campaign_id, segment_id, channel_id, day, stage)**

### Schema

```json
[
  {
    "event_id": "FNL_0001",
    "timestamp": "2026-03-14T00:00:00",
    "date": "2026-03-14",
    "campaign_id": "CAMP_001",
    "segment_id": "SEG_001",
    "channel_id": "CH_01",
    "stage": "visit",
    "count": 1200,
    "source": "web_analytics",
    "confidence": 0.9
  }
]
```

### Fields

* `event_id` (string, unique)
* `timestamp` (ISO string)
* `date` (YYYY-MM-DD) *(optional but helpful for grouping)*
* `campaign_id` (FK → campaigns)
* `segment_id` (FK → segments)
* `channel_id` (FK → channels)
* `stage` (enum): `impression | click | visit | signup | demo_request | purchase | feature_adopt`
* `count` (int)
* `source` (enum): `ads_platform | crm | web_analytics | product_analytics`
* `confidence` (0–1 float) *(lets you do “trust gating” later)*

### What it enables

* Stage drop-off detection: “strong CTR, weak signup”
* Segment journey differences: “SEG_002 converts late; keep nurturing”
* Campaign health by funnel stage in your exec report

---

## 2) `budget_actions.json` (controlled reallocations)

### Why add it

You already track ROI by campaign  and channel cost models . Budget actions add one critical dimension: **the orchestrator can move money with an audit trail**.

### Grain

**Budget movement event**

### Schema

```json
[
  {
    "budget_action_id": "BUD_0001",
    "timestamp": "2026-03-16T10:30:00",
    "campaign_id": "CAMP_002",
    "from_channel_id": "CH_03",
    "to_channel_id": "CH_02",
    "amount": 1500,
    "currency": "USD",
    "reason_code": "roi_optimization",
    "reason_detail": "Paid search variant outperforming social on demo_request_rate",
    "proposed_by": "orchestrator",
    "approved_by_human": true,
    "approver_id": "HUMAN_01",
    "approval_latency_minutes": 55,
    "status": "executed"
  }
]
```

### Fields

* `budget_action_id` (string, unique)
* `timestamp`
* `campaign_id` (FK)
* `from_channel_id`, `to_channel_id` (FK)
* `amount` (float)
* `currency` (string, default "USD")
* `reason_code` (enum): `roi_optimization | risk_mitigation | scale_winner | stop_loss | capacity_constraint`
* `reason_detail` (string)
* `proposed_by` (enum): `orchestrator | human`
* `approved_by_human` (bool)
* `approver_id` (string nullable)
* `approval_latency_minutes` (int nullable)
* `status` (enum): `proposed | approved | rejected | executed | rolled_back`

### What it enables

* CEO-friendly “capital allocation decisions”
* Governance: prove approvals happened before spend shifted
* Easy “before/after” analysis tied to decisions

---

## 3) `campaign_risk_signals.json` (risk & escalation layer)

### Why add it

You already log decisions and sometimes require human override . Risk signals are the missing glue that turns “override happened” into a structured *risk reason + severity*.

### Grain

**Risk signal event**

### Schema

```json
[
  {
    "risk_id": "RSK_0001",
    "timestamp": "2026-03-15T10:00:00",
    "campaign_id": "CAMP_002",
    "experiment_id": "EXP_004",
    "asset_ids": ["ASSET_007", "ASSET_008"],
    "risk_type": "high_clicks_zero_conversions",
    "severity": "high",
    "evidence": {
      "impressions": 28500,
      "clicks": 2140,
      "conversions": 0
    },
    "recommended_action": "human_review",
    "status": "open"
  }
]
```

### Fields

* `risk_id` (unique)
* `timestamp`
* `campaign_id` (FK)
* `experiment_id` (FK nullable)
* `asset_ids` (list of asset IDs)
* `risk_type` (enum):
  `brand_safety | policy_violation | low_conversion | high_clicks_zero_conversions | spend_spike | data_quality | segment_mismatch`
* `severity` (enum): `low | medium | high | critical`
* `evidence` (object, flexible)
* `recommended_action` (enum): `pause | reduce_budget | scale_down | human_review | revise_copy | investigate_tracking`
* `status` (enum): `open | in_review | mitigated | dismissed`

### What it enables

* A clean “risk inbox”
* Escalation thresholds: severity + confidence + spend
* Report section: “Top risks this week”

---

## 4) `segment_rollups.json` (executive segment intelligence)

### Why add it

You already have segment definitions  and performance metrics , but no compact summary layer. Rollups make your exec report faster to compute and easier to read.

### Grain

**(campaign_id, segment_id, window_end_date)**

### Schema

```json
[
  {
    "rollup_id": "SEGROLL_0001",
    "window_start": "2026-03-10",
    "window_end": "2026-03-16",
    "campaign_id": "CAMP_001",
    "segment_id": "SEG_002",
    "impressions": 18100,
    "clicks": 1150,
    "conversions": 156,
    "spend": 346.0,
    "revenue_proxy": 7850.0,
    "primary_kpi": "conversion_rate",
    "kpi_value": 0.0086,
    "trend_vs_prior_window": 0.12,
    "recommendation": "scale"
  }
]
```

### Fields

* `rollup_id` (unique)
* `window_start`, `window_end` (YYYY-MM-DD)
* `campaign_id` (FK)
* `segment_id` (FK)
* `impressions`, `clicks`, `conversions` (int)
* `spend` (float)
* `revenue_proxy` (float)
* `primary_kpi` (string; should mirror campaign KPI)
* `kpi_value` (float)
* `trend_vs_prior_window` (float; -1.0 to +1.0 is fine)
* `recommendation` (enum): `scale | hold | revise | stop`

### What it enables

* Segment-level “what’s working”
* Easy prioritization aligned to segment priority tiers
* Inputs to budget reallocation logic

---

# Join keys and relationships

These are the only join paths you need:

* **Campaign-centric joins**

  * `campaign_id` is the backbone across: campaigns , experiments , ROI ledger , decisions , plus new: funnel events, budget actions, risk signals, rollups.

* **Segment-centric joins**

  * `segment_id` links segments  ↔ assets  ↔ new funnel/rollups.

* **Channel-centric joins**

  * `channel_id` links channels  ↔ assets  ↔ funnel events ↔ budget actions.

* **Experiment + asset traceability**

  * experiments  tie to assets ; performance  ties to experiment_id + asset_id.

---

# Minimal V2 “contract” (data constraints that keep MVP sane)

These rules keep your datasets realistic but lightweight:

1. **Every campaign has at least 1 experiment** (already true)
2. **Every experiment maps to exactly 2 assets** (control + variant)
3. **Performance metrics are asset+experiment snapshots** (already true)
4. **Budget actions require human approval when amount ≥ X** (simple governance rule)
5. **Risk signals can open → force human review** (ties to `human_override` patterns you already track)

---

# Optional “nice but still MVP” add-on (if you want one more)

## `attribution_hints.json` (NOT full attribution)

Just store “self-reported” or “first-touch” hints for realism:

```json
[
  {
    "hint_id": "ATTR_0001",
    "timestamp": "2026-03-16T12:00:00",
    "campaign_id": "CAMP_001",
    "channel_id": "CH_01",
    "attributed_conversions": 22,
    "method": "last_touch",
    "confidence": 0.6
  }
]
```

Only add this if you want to talk about attribution *without building attribution*.

---

# What I recommend you implement first (V2 MVP cut)

If you want the cleanest V2 MVP:

✅ `funnel_events.json`
✅ `budget_actions.json`
✅ `campaign_risk_signals.json`
✅ keep `segment_rollups.json` optional (you can compute it from performance later)




# funnel_events.json

In [ ]:
[
  {
    "event_id": "FNL_0001",
    "timestamp": "2026-03-14T00:00:00",
    "date": "2026-03-14",
    "campaign_id": "CAMP_001",
    "segment_id": "SEG_001",
    "channel_id": "CH_01",
    "stage": "visit",
    "count": 1200,
    "source": "web_analytics",
    "confidence": 0.92
  },
  {
    "event_id": "FNL_0002",
    "timestamp": "2026-03-14T00:00:00",
    "date": "2026-03-14",
    "campaign_id": "CAMP_001",
    "segment_id": "SEG_001",
    "channel_id": "CH_01",
    "stage": "signup",
    "count": 180,
    "source": "crm",
    "confidence": 0.90
  },
  {
    "event_id": "FNL_0003",
    "timestamp": "2026-03-14T00:00:00",
    "date": "2026-03-14",
    "campaign_id": "CAMP_001",
    "segment_id": "SEG_002",
    "channel_id": "CH_01",
    "stage": "visit",
    "count": 950,
    "source": "web_analytics",
    "confidence": 0.91
  },
  {
    "event_id": "FNL_0004",
    "timestamp": "2026-03-14T00:00:00",
    "date": "2026-03-14",
    "campaign_id": "CAMP_001",
    "segment_id": "SEG_002",
    "channel_id": "CH_01",
    "stage": "signup",
    "count": 140,
    "source": "crm",
    "confidence": 0.89
  },
  {
    "event_id": "FNL_0005",
    "timestamp": "2026-03-15T00:00:00",
    "date": "2026-03-15",
    "campaign_id": "CAMP_002",
    "segment_id": "SEG_002",
    "channel_id": "CH_02",
    "stage": "visit",
    "count": 620,
    "source": "web_analytics",
    "confidence": 0.87
  },
  {
    "event_id": "FNL_0006",
    "timestamp": "2026-03-15T00:00:00",
    "date": "2026-03-15",
    "campaign_id": "CAMP_002",
    "segment_id": "SEG_002",
    "channel_id": "CH_02",
    "stage": "demo_request",
    "count": 72,
    "source": "crm",
    "confidence": 0.88
  },
  {
    "event_id": "FNL_0007",
    "timestamp": "2026-03-15T00:00:00",
    "date": "2026-03-15",
    "campaign_id": "CAMP_002",
    "segment_id": "SEG_003",
    "channel_id": "CH_03",
    "stage": "visit",
    "count": 480,
    "source": "web_analytics",
    "confidence": 0.85
  },
  {
    "event_id": "FNL_0008",
    "timestamp": "2026-03-15T00:00:00",
    "date": "2026-03-15",
    "campaign_id": "CAMP_002",
    "segment_id": "SEG_003",
    "channel_id": "CH_03",
    "stage": "demo_request",
    "count": 0,
    "source": "crm",
    "confidence": 0.80
  },
  {
    "event_id": "FNL_0009",
    "timestamp": "2026-03-16T00:00:00",
    "date": "2026-03-16",
    "campaign_id": "CAMP_003",
    "segment_id": "SEG_005",
    "channel_id": "CH_04",
    "stage": "visit",
    "count": 410,
    "source": "product_analytics",
    "confidence": 0.90
  },
  {
    "event_id": "FNL_0010",
    "timestamp": "2026-03-16T00:00:00",
    "date": "2026-03-16",
    "campaign_id": "CAMP_003",
    "segment_id": "SEG_005",
    "channel_id": "CH_04",
    "stage": "feature_adopt",
    "count": 0,
    "source": "product_analytics",
    "confidence": 0.88
  }
]


# budget_actions.json

In [ ]:
[
  {
    "budget_action_id": "BUD_0001",
    "timestamp": "2026-03-16T10:30:00",
    "campaign_id": "CAMP_002",
    "from_channel_id": "CH_03",
    "to_channel_id": "CH_02",
    "amount": 1500.00,
    "currency": "USD",
    "reason_code": "scale_winner",
    "reason_detail": "Paid search variants outperforming social for demo requests",
    "proposed_by": "orchestrator",
    "approved_by_human": true,
    "approver_id": "HUMAN_01",
    "approval_latency_minutes": 55,
    "status": "executed"
  },
  {
    "budget_action_id": "BUD_0002",
    "timestamp": "2026-03-17T09:10:00",
    "campaign_id": "CAMP_001",
    "from_channel_id": "CH_01",
    "to_channel_id": "CH_01",
    "amount": 2000.00,
    "currency": "USD",
    "reason_code": "scale_winner",
    "reason_detail": "Email variants driving above-target signup conversion rate",
    "proposed_by": "orchestrator",
    "approved_by_human": true,
    "approver_id": "HUMAN_02",
    "approval_latency_minutes": 40,
    "status": "executed"
  },
  {
    "budget_action_id": "BUD_0003",
    "timestamp": "2026-03-18T11:45:00",
    "campaign_id": "CAMP_003",
    "from_channel_id": "CH_04",
    "to_channel_id": "CH_02",
    "amount": 1000.00,
    "currency": "USD",
    "reason_code": "stop_loss",
    "reason_detail": "In-app feature campaign stalled with zero downstream adoption",
    "proposed_by": "orchestrator",
    "approved_by_human": false,
    "approver_id": null,
    "approval_latency_minutes": null,
    "status": "proposed"
  },
  {
    "budget_action_id": "BUD_0004",
    "timestamp": "2026-03-18T14:20:00",
    "campaign_id": "CAMP_002",
    "from_channel_id": "CH_02",
    "to_channel_id": "CH_03",
    "amount": 500.00,
    "currency": "USD",
    "reason_code": "risk_mitigation",
    "reason_detail": "Social channel flagged for human review after high CTR but zero conversions",
    "proposed_by": "orchestrator",
    "approved_by_human": true,
    "approver_id": "HUMAN_03",
    "approval_latency_minutes": 75,
    "status": "executed"
  },
  {
    "budget_action_id": "BUD_0005",
    "timestamp": "2026-03-19T08:50:00",
    "campaign_id": "CAMP_001",
    "from_channel_id": "CH_02",
    "to_channel_id": "CH_01",
    "amount": 1200.00,
    "currency": "USD",
    "reason_code": "roi_optimization",
    "reason_detail": "Email channel showing lower CPA than paid search for trial signups",
    "proposed_by": "orchestrator",
    "approved_by_human": true,
    "approver_id": "HUMAN_01",
    "approval_latency_minutes": 32,
    "status": "executed"
  },
  {
    "budget_action_id": "BUD_0006",
    "timestamp": "2026-03-19T13:15:00",
    "campaign_id": "CAMP_002",
    "from_channel_id": "CH_03",
    "to_channel_id": "CH_03",
    "amount": 800.00,
    "currency": "USD",
    "reason_code": "capacity_constraint",
    "reason_detail": "Limited paid search inventory; reallocating additional social spend",
    "proposed_by": "human",
    "approved_by_human": true,
    "approver_id": "HUMAN_04",
    "approval_latency_minutes": 20,
    "status": "executed"
  }
]


# campaign_risk_signals.json

In [ ]:
[
  {
    "risk_id": "RSK_0001",
    "timestamp": "2026-03-15T10:05:00",
    "campaign_id": "CAMP_002",
    "experiment_id": "EXP_004",
    "asset_ids": ["ASSET_007", "ASSET_008"],
    "risk_type": "high_clicks_zero_conversions",
    "severity": "high",
    "evidence": {
      "impressions": 28500,
      "clicks": 2140,
      "conversions": 0
    },
    "recommended_action": "human_review",
    "status": "open"
  },
  {
    "risk_id": "RSK_0002",
    "timestamp": "2026-03-17T13:40:00",
    "campaign_id": "CAMP_003",
    "experiment_id": "EXP_005",
    "asset_ids": ["ASSET_009", "ASSET_010"],
    "risk_type": "low_conversion",
    "severity": "critical",
    "evidence": {
      "visits": 830,
      "feature_clicks": 0
    },
    "recommended_action": "pause",
    "status": "open"
  },
  {
    "risk_id": "RSK_0003",
    "timestamp": "2026-03-14T16:20:00",
    "campaign_id": "CAMP_001",
    "experiment_id": "EXP_001",
    "asset_ids": ["ASSET_001", "ASSET_002"],
    "risk_type": "spend_spike",
    "severity": "medium",
    "evidence": {
      "prior_daily_spend": 380,
      "current_daily_spend": 640
    },
    "recommended_action": "investigate_tracking",
    "status": "mitigated"
  },
  {
    "risk_id": "RSK_0004",
    "timestamp": "2026-03-18T09:15:00",
    "campaign_id": "CAMP_002",
    "experiment_id": "EXP_003",
    "asset_ids": ["ASSET_005", "ASSET_006"],
    "risk_type": "segment_mismatch",
    "severity": "medium",
    "evidence": {
      "segment_id": "SEG_002",
      "expected_demo_rate": 0.015,
      "actual_demo_rate": 0.006
    },
    "recommended_action": "revise_copy",
    "status": "in_review"
  },
  {
    "risk_id": "RSK_0005",
    "timestamp": "2026-03-19T11:50:00",
    "campaign_id": "CAMP_001",
    "experiment_id": "EXP_002",
    "asset_ids": ["ASSET_003", "ASSET_004"],
    "risk_type": "brand_safety",
    "severity": "low",
    "evidence": {
      "flagged_phrase": "Cut your monthly bill in half",
      "policy_rule": "ABSOLUTE_SAVINGS_CLAIM"
    },
    "recommended_action": "human_review",
    "status": "dismissed"
  },
  {
    "risk_id": "RSK_0006",
    "timestamp": "2026-03-19T15:05:00",
    "campaign_id": "CAMP_002",
    "experiment_id": null,
    "asset_ids": [],
    "risk_type": "data_quality",
    "severity": "low",
    "evidence": {
      "missing_events": 42,
      "source": "web_analytics"
    },
    "recommended_action": "investigate_tracking",
    "status": "open"
  }
]


# segment_rollups.json

In [ ]:
[
  {
    "rollup_id": "SEGROLL_0001",
    "window_start": "2026-03-10",
    "window_end": "2026-03-16",
    "campaign_id": "CAMP_001",
    "segment_id": "SEG_001",
    "impressions": 16200,
    "clicks": 1040,
    "conversions": 122,
    "spend": 324.00,
    "revenue_proxy": 6400.00,
    "primary_kpi": "conversion_rate",
    "kpi_value": 0.0075,
    "trend_vs_prior_window": 0.08,
    "recommendation": "hold"
  },
  {
    "rollup_id": "SEGROLL_0002",
    "window_start": "2026-03-10",
    "window_end": "2026-03-16",
    "campaign_id": "CAMP_001",
    "segment_id": "SEG_002",
    "impressions": 18100,
    "clicks": 1150,
    "conversions": 156,
    "spend": 346.00,
    "revenue_proxy": 7850.00,
    "primary_kpi": "conversion_rate",
    "kpi_value": 0.0086,
    "trend_vs_prior_window": 0.12,
    "recommendation": "scale"
  },
  {
    "rollup_id": "SEGROLL_0003",
    "window_start": "2026-03-10",
    "window_end": "2026-03-16",
    "campaign_id": "CAMP_002",
    "segment_id": "SEG_002",
    "impressions": 9200,
    "clicks": 610,
    "conversions": 52,
    "spend": 915.00,
    "revenue_proxy": 2600.00,
    "primary_kpi": "demo_request_rate",
    "kpi_value": 0.0057,
    "trend_vs_prior_window": -0.05,
    "recommendation": "revise"
  },
  {
    "rollup_id": "SEGROLL_0004",
    "window_start": "2026-03-10",
    "window_end": "2026-03-16",
    "campaign_id": "CAMP_002",
    "segment_id": "SEG_003",
    "impressions": 15200,
    "clicks": 990,
    "conversions": 0,
    "spend": 228.00,
    "revenue_proxy": 0.00,
    "primary_kpi": "demo_request_rate",
    "kpi_value": 0.0000,
    "trend_vs_prior_window": -0.22,
    "recommendation": "stop"
  },
  {
    "rollup_id": "SEGROLL_0005",
    "window_start": "2026-03-10",
    "window_end": "2026-03-16",
    "campaign_id": "CAMP_003",
    "segment_id": "SEG_005",
    "impressions": 8600,
    "clicks": 640,
    "conversions": 0,
    "spend": 0.00,
    "revenue_proxy": 0.00,
    "primary_kpi": "feature_click_through_rate",
    "kpi_value": 0.0000,
    "trend_vs_prior_window": -0.30,
    "recommendation": "stop"
  },
  {
    "rollup_id": "SEGROLL_0006",
    "window_start": "2026-03-10",
    "window_end": "2026-03-16",
    "campaign_id": "CAMP_002",
    "segment_id": "SEG_001",
    "impressions": 5100,
    "clicks": 290,
    "conversions": 18,
    "spend": 410.00,
    "revenue_proxy": 900.00,
    "primary_kpi": "demo_request_rate",
    "kpi_value": 0.0035,
    "trend_vs_prior_window": 0.02,
    "recommendation": "hold"
  }
]


# attribution_hints.json

In [ ]:
[
  {
    "hint_id": "ATTR_0001",
    "timestamp": "2026-03-16T12:00:00",
    "campaign_id": "CAMP_001",
    "channel_id": "CH_01",
    "attributed_conversions": 28,
    "method": "last_touch",
    "confidence": 0.65
  },
  {
    "hint_id": "ATTR_0002",
    "timestamp": "2026-03-16T12:00:00",
    "campaign_id": "CAMP_001",
    "channel_id": "CH_02",
    "attributed_conversions": 14,
    "method": "assist",
    "confidence": 0.45
  },
  {
    "hint_id": "ATTR_0003",
    "timestamp": "2026-03-17T10:30:00",
    "campaign_id": "CAMP_002",
    "channel_id": "CH_02",
    "attributed_conversions": 19,
    "method": "last_touch",
    "confidence": 0.60
  },
  {
    "hint_id": "ATTR_0004",
    "timestamp": "2026-03-17T10:30:00",
    "campaign_id": "CAMP_002",
    "channel_id": "CH_03",
    "attributed_conversions": 5,
    "method": "assist",
    "confidence": 0.40
  },
  {
    "hint_id": "ATTR_0005",
    "timestamp": "2026-03-18T09:15:00",
    "campaign_id": "CAMP_003",
    "channel_id": "CH_04",
    "attributed_conversions": 0,
    "method": "last_touch",
    "confidence": 0.70
  },
  {
    "hint_id": "ATTR_0006",
    "timestamp": "2026-03-18T15:45:00",
    "campaign_id": "CAMP_001",
    "channel_id": "CH_03",
    "attributed_conversions": 3,
    "method": "first_touch",
    "confidence": 0.35
  }
]
